## One of the early breakthroughs in reinforcement learning was the development of an off-policy TD control algorithm known as Q-learning (Watkins, 1989)

##  Installation of required tools -
We will use Open AI's gym environment for this tutorial.

In [75]:
! pip install git+https://github.com/openai/gym.git

  Cloning https://github.com/openai/gym.git to /tmp/pip-req-build-YaHM3a
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /tmp/pip-ephem-wheel-cache-zWgL14/wheels/13/4b/2f/79d47b11ac3a37fc33b74d4bdf9031be155b70ff8b7ca24571
Successfully built gym


## Task Description

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. 

The water is mostly frozen, but there are a **few holes** where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.

Consider a **2-D grid world**. The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.  Let us visualize this environment https://gym.openai.com/envs/FrozenLake-v0/

SFFF       (S: starting point, safe)

FHFH       (F: frozen surface, safe)

FFFH       (H: hole, fall to your doom)

HFFG       (G: goal, where the frisbee is located)

## Environments 

Where do these environments come from ? 
- [Arcade Learning Environment](https://github.com/mgbellemare/Arcade-Learning-Environment)
- [ Open AI Gym](https://gym.openai.com/)
- [Deep Mind Lab](https://github.com/deepmind/lab)

**Problem solving approach in RL**

* Task : what do you want to solve ? 
* Environment : determinstic or stochastic 
* Transition dynamics : P(s' | s ),   R( s, a)




## Understanding what is the action space and  observation space

In [76]:
import numpy as np
import gym

env = gym.make('FrozenLake-v0')

print(env.action_space)

print(env.observation_space)

Discrete(4)
Discrete(16)


In [77]:
env.render()


SFFF
FHFH
FFFH
HFFG


Notice that 


*   0 Left 
*   1 Down
*   2 Right 
*   3 Up


In [78]:
action_set = [0, 1, 2, 3]
env.reset()
a,b,c,d = env.step(3)
env.render()

  (Up)
SFFF
FHFH
FFFH
HFFG


In [79]:
action_set = [0, 1, 2, 3]
env.reset()
env.render()
obs,rew,done,info = env.step(2)
env.render()


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


## The environment’s step function

Taking a step returns four values. These are:

* **observation (object)**: an environment-specific object representing your observation of the environment. For example, pixel data from a camera, joint angles and joint velocities of a robot, or the board state in a board game.


* **reward (float)**: amount of reward achieved by the previous action. The scale varies between environments, but the goal is always to increase your total reward.

* **done (boolean)**: whether it’s time to reset the environment again. Most (but not all) tasks are divided up into well-defined episodes, and done being True indicates the episode has terminated. (For example, perhaps the pole tipped too far, or you lost your last life.)

* **info (dict)**: diagnostic information useful for debugging. It can sometimes be useful for learning (for example, it might contain the raw probabilities behind the environment’s last state change). However, official evaluations of your agent are not allowed to use this for learning.

# Sample Agent Environment Interface 

In [80]:
for i_episode in range(2):
    observation = env.reset()
    for t in range(10):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break


SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
4
  (Left)
SFFF
FHFH
FFFH
HFFG
8
  (Up)
SFFF
FHFH
FFFH
HFFG
4
  (Up)
SFFF
FHFH
FFFH
HFFG
4
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0

SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
1
  (Up)
SFFF
FHFH
FFFH
HFFG
1
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Right)
SFFF
FHFH
FFFH
HFFG
2
  (Up)
SFFF
FHFH
FFFH
HFFG
2


## Epsilon Greedy Function 

    - Chooses a greedy action most of the time but with a probability eps chooses a random action
    - Chooses random action with probability of eps; argmax Q(s, .) with probability of (1-eps)

In [0]:
def eps_greedy(q_vals, eps, state):
    """
    Inputs:
        q_vals: q value tables
        eps: epsilon
        state: current state
    Outputs:
        random action with probability of eps; argmax Q(s, .) with probability of (1-eps)
    """
    import random
    if random.random() <= eps:
        action = env.action_space.sample() # sample an action randomly # sample an action randomly
    else:
        action = np.argmax(q_vals[state,:])
    return action

## Q learning update function. 

* Q learning update function. After we observe a transition $s, a, s', r$,
     
     $$\textrm{target}(s') = R(s,a,s') + \gamma \max_{a'} Q_{\theta_k}(s',a')$$
     $$\textrm{delta}(s') = \textrm{target}(s') - Q_{\theta_k}(s',a')
     $$$$Q_{k+1}(s,a) \leftarrow Q_k(s,a) + \alpha * \left( \textrm{delta}(s') \right)$$


In [0]:
def q_learning_update(gamma, alpha, q_vals, cur_state, action, next_state, reward):
    """
    Inputs:
        gamma: discount factor
        alpha: learning rate
        q_vals: q value table
        cur_state: current state
        action: action taken in current state
        next_state: next state results from taking `action` in `cur_state`
        reward: reward received from this transition
    
    Performs in-place update of q_vals table to implement one step of Q-learning
    """
    delta = reward + gamma * np.max(q_vals[next_state,:]) - q_vals[cur_state,action]
    q_vals[cur_state,action] = q_vals[cur_state,action] + alpha * delta

## Algorithm

**Let us first see what happens when we always take Greedy Action**

In [113]:
env = gym.make('FrozenLake-v0')
#env = gym.make('FrozenLakeNotSlippery18x18-v0')

Q = np.zeros([env.observation_space.n,env.action_space.n])
gamma = 0.95
alpha = 0.8
epsilon = 0.1
episodes_num = 2000
rList = []
for itr in range(episodes_num):
    cur_state = env.reset()
    ret = 0
    done = False
    while not done:
        action = eps_greedy(Q, epsilon, cur_state)
        next_state, reward, done, info = env.step(action)
        q_learning_update(gamma, alpha, Q, cur_state, action, next_state, reward)
        cur_state = next_state
        ret+=reward
    rList.append(ret)
print ("Score over time: " +  str(sum(rList)/episodes_num))
print("Q-values: %s" %Q)

Score over time: 0.0
Q-values: [[ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]
 [ 0.00000  0.00000  0.00000  0.00000]]


**Do something else**

In [114]:
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
env = gym.make('FrozenLake-v0')

Q = np.zeros([env.observation_space.n,env.action_space.n])
gamma = 0.98 #0.95 
alpha = 0.8
epsilon = 0.6
episodes_num = 2000
rList = []
for itr in range(episodes_num):
    cur_state = env.reset()
    ret = 0
    done = False
    while not done:
        #action = eps_greedy(Q, epsilon, cur_state)
        #print(action)
        action = np.argmax(Q[cur_state,:] + np.random.randn(1,env.action_space.n)*(1./(itr+1)))
        next_state, reward, done, info = env.step(action)
        q_learning_update(gamma, alpha, Q, cur_state, action, next_state, reward)
        #Q[cur_state,action] = Q[cur_state,action] + alpha*(reward + gamma*np.max(Q[next_state,:]) - Q[cur_state,action])
        cur_state = next_state
        ret+=reward
    rList.append(ret)
    #epsilon = max(epsilon-0.002,0.1)
print ("Score over time: " +  str(sum(rList)/episodes_num))
print("Q-values:", Q)

Score over time: 0.5705
('Q-values:', array([[ 0.37802,  0.00524,  0.00533,  0.00314],
       [ 0.00024,  0.00276,  0.00082,  0.34199],
       [ 0.00134,  0.28442,  0.00135,  0.00248],
       [ 0.00020,  0.00000,  0.00000,  0.26694],
       [ 0.30730,  0.00330,  0.00151,  0.00027],
       [ 0.00000,  0.00000,  0.00000,  0.00000],
       [ 0.00029,  0.00006,  0.00009,  0.00002],
       [ 0.00000,  0.00000,  0.00000,  0.00000],
       [ 0.00105,  0.00002,  0.00106,  0.27389],
       [ 0.00000,  0.78918,  0.00086,  0.00000],
       [ 0.78100,  0.00000,  0.00004,  0.00042],
       [ 0.00000,  0.00000,  0.00000,  0.00000],
       [ 0.00000,  0.00000,  0.00000,  0.00000],
       [ 0.00065,  0.00051,  0.64461,  0.00032],
       [ 0.00000,  0.95145,  0.00000,  0.00000],
       [ 0.00000,  0.00000,  0.00000,  0.00000]]))


In [112]:
print(len(rList))

AttributeError: ignored

In [128]:
print(rList[3])

0.0


In [142]:
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt #display purposes.

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 5))
plt.plot(1, color="royalblue");
plt.xlim((0, len(rList))
plt.xlabel("Time Step")




SyntaxError: ignored